Import Libs

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sentence_transformers import SentenceTransformer

# own packages
from preprocess_data import clean_text
from preprocess_data import lemmatize_text
from models import model_sfnn_train
from models import model_rf_train, model_naives_bayes, model_logistic_regression, predict_values, model_xgboost



## Importing and reading the data

In [3]:
# get csv data
def read_Data(path):

    # Define column names
    column_names = ['label', 'text']

    df = pd.read_csv(path, delimiter='\t', encoding="utf-8-sig", header=None, names=column_names)
    df.columns = df.columns.str.replace('\ufeff', '')
    df['label'] = df['label'].astype(str).str.replace('\ufeff', '')
    df['label'] = df['label'].astype(int)
    
    return df



In [4]:
## read data windows
#data_train_val = read_Data(r".\data\training_data_lowercase.csv")           # used for train and val data
#data_test = read_Data(r".\data\testing_data_lowercase_nolabels.csv")        # data for the predictions

## read data Mac
data_train_val = read_Data("data/training_data_lowercase.csv")           # used for train and val data
data_test = read_Data("data/testing_data_lowercase_nolabels.csv")        # data for the predictions


## split data_train into train and val data
data_train, data_val = train_test_split(data_train_val, test_size=0.08, random_state=42, stratify=data_train_val['label'])


In [5]:
## preview data
#print(data_train.head, "\n")
#print(data_test.head, "\n")
print("Shape train data:\n", data_train_val.shape)
print("Shape test data:\n", data_test.shape)


Shape train data:
 (34152, 2)
Shape test data:
 (9984, 2)


## Pre-processing data

Setup with advanced data cleaning

In [6]:
# Preprocessing steps

# clean data
data_train['cleaned_text'] = data_train['text'].apply(clean_text)
data_val['cleaned_text'] = data_val['text'].apply(clean_text)
data_test['cleaned_text'] = data_test['text'].apply(clean_text)

print("Cleand text train: \n", data_train["cleaned_text"].head, "\n")
print("Cleand text test: \n", data_test["cleaned_text"].head, "\n")


## lemmatize data
data_train['lemmatized_text'] = data_train['cleaned_text'].apply(lemmatize_text)
data_val['lemmatized_text'] = data_val['cleaned_text'].apply(lemmatize_text)
data_test['lemmatized_text'] = data_test['cleaned_text'].apply(lemmatize_text)

print("lemmatized_text train: \n", data_train['lemmatized_text'].head, "\n")
print("lemmatized_text test: \n", data_test['lemmatized_text'].head, "\n")

# calc tf-idf matrix on lemmatized data
vectorizer = TfidfVectorizer(ngram_range=(1,2))

tfidf_matrix_train = vectorizer.fit_transform(data_train['lemmatized_text'])
tfidf_matrix_val = vectorizer.transform(data_val['lemmatized_text'])
tfidf_matrix_test = vectorizer.transform(data_test['lemmatized_text'])


Cleand text train: 
 <bound method NDFrame.head of 6602     [fox, host, throws, hissy, fit, cant, call, pr...
930      [trump, unpopular, crowds, bussed, cheer, pola...
29744    [congratulations, eu, moves, brexit, phase, tw...
24734    [trumps, national, security, adviser, vows, ta...
15772    [republican, turns, tables, fbi, deputy, direc...
                               ...                        
27469    [us, pledges, strong, response, event, another...
27793    [republican, presidential, candidate, cruz, ra...
883                                         [like, father]
25547    [congress, passes, funding, bill, averting, go...
3939     [hilariously, amazing, video, weve, waiting, e...
Name: cleaned_text, Length: 31419, dtype: object> 

Cleand text test: 
 <bound method NDFrame.head of 0       [copycat, muslim, terrorist, arrested, assault...
1       [wow, chicago, protester, caught, camera, admi...
2       [germanys, fdp, look, fill, schaeubles, big, s...
3       [mi, school, sen

**TF-IDF only**

With and without stop-words function

Pre-processing setup 2: TF-IDF Vectorizer (without stopwords argument)

In [7]:
# calc tf-idf matrix with uncleaned data
vectorizer2 = TfidfVectorizer(ngram_range=(1, 3))
X_train_tfidf = vectorizer2.fit_transform(data_train['text'])
X_val_tfidf = vectorizer2.transform(data_val['text'])
X_test_tfidf = vectorizer2.transform(data_test['text'])


Pre-processing setup 3: TF-IDF Vectorizer (without stopwords argument)

In [8]:

# calc tf-idf matrix with uncleaned data, tf-idf removing stopwords
vectorizer3 = TfidfVectorizer(stop_words="english", ngram_range=(1, 3))
X_train_tfidf3 = vectorizer3.fit_transform(data_train['text'])
X_val_tfidf3 = vectorizer3.transform(data_val['text'])
X_test_tfidf3 = vectorizer3.transform(data_test['text'])

Pre-processing setup 4: Embeddings

In [9]:
# Load pretrained embedding model
embedder = SentenceTransformer("all-MiniLM-L6-v2")

# Convert text to embeddings
X_train_emb = embedder.encode(data_train['text'].tolist(), show_progress_bar=True)
X_val_emb = embedder.encode(data_val['text'].tolist(), show_progress_bar=True)
X_test_emb = embedder.encode(data_test['text'].tolist(), show_progress_bar=True)

d:\ownCloud\Dokumente\Ironhack\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Georg\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back

## Applying and evaluating models

### Naive Bayes Model

In [ ]:
# Preprocessing setup 1: calc tf-idf on lemmatized data
model_nb1 = model_naives_bayes(tfidf_matrix_train, data_train['label'], tfidf_matrix_val, data_val['label'])


In [11]:
# Preprocessing setup 2: tf-idf only
model_nb2 = model_naives_bayes(X_train_tfidf, data_train['label'], X_val_tfidf, data_val['label'])


Train Accuracy:1.00

Validation Accuracy: 0.95

classification_report:
               precision    recall  f1-score   support

           0     0.9383    0.9623    0.9501      1406
           1     0.9589    0.9329    0.9458      1327

    accuracy                         0.9480      2733
   macro avg     0.9486    0.9476    0.9480      2733
weighted avg     0.9483    0.9480    0.9480      2733



In [ ]:
# Preprocessing setup 3: tf-idf removing stopwords
model_nb3 = model_naives_bayes(X_train_tfidf3, data_train['label'], X_val_tfidf3, data_val['label'])


### XGBoost

In [ ]:
# XGBoost Setup 1
model_xgb1 = model_xgboost(tfidf_matrix_train, data_train['label'], tfidf_matrix_val, data_val['label'])

In [ ]:
# XGBoost Setup 2
model_xgb2 = model_xgboost(X_train_tfidf, data_train['label'], X_val_tfidf, data_val['label'])

In [ ]:
# XGBoost Setup 3
model_xgb3 = model_xgboost(X_train_tfidf3, data_train['label'], X_val_tfidf3, data_val['label'])

### Random Forest

In [ ]:
# Random Forest setup 1
model_rf1 = model_rf_train(tfidf_matrix_train, tfidf_matrix_val, data_train['label'], data_val['label'])

In [ ]:
# Random Forest setup 2
model_rf2 = model_rf_train(X_train_tfidf, X_val_tfidf, data_train['label'], data_val['label'])

In [ ]:
# Random Forest setup 2
model_rf3 = model_rf_train(X_train_tfidf3, X_val_tfidf3, data_train['label'], data_val['label'])

### Logistic Regression

In [ ]:
#Logistic Regression setup1
model_lr1 = model_logistic_regression(tfidf_matrix_train, data_train['label'], tfidf_matrix_val, data_val['label'])

In [ ]:
#Logistic Regression setup2
model_lr2 = model_logistic_regression(X_train_tfidf, data_train['label'], X_val_tfidf, data_val['label'])

In [ ]:
#Logistic Regression setup3
model_lr3 = model_logistic_regression(X_train_tfidf3, data_train['label'], X_val_tfidf3, data_val['label'])

In [ ]:
# Preprocessing setup 4: Embeddings
model_lr4 = model_logistic_regression(X_train_emb, data_train['label'], X_val_emb, data_val['label'])

### Simple Feedforward Neural Network

In [ ]:
# Simple Feedforward NN with preprocessing setup2
model_sfnn2 = model_sfnn_train(X_train_tfidf, data_train['label'], X_val_tfidf, data_val['label'])

In [ ]:
# Simple Feedforward NN with preprocessing setup3
model_sfnn3 = model_sfnn_train(X_train_tfidf3, data_train['label'], X_val_tfidf3, data_val['label'])

In [25]:
## make predictions on test data
filepath = r".\data\testing_data_lowercase_labels.csv"

#predict_values(model_rf, tfidf_matrix_test, data_test, filepath)
predict_values(model_nb2, X_test_tfidf, data_test, filepath)


#  model_nb2 = model_naives_bayes(X_train_tfidf, data_train['label'], X_val_tfidf, data_val['label'])

✅ Predictions saved to: .\data\testing_data_lowercase_labels.csv
